# Process Performance and Reliability Data Sets



In [1]:
import sys
import pandas as pd 
sys.path.append('..')
from Model.Fleet import Fleet

avg_path = '../southernoperations/AIID/AVG'
compressed_path = '../southernoperations/AIID/CMP'
yaw_path = '../southernoperations/AIID/YAW'
oem_powercurves_path = '../assets/data/power_curves/all_power_curves.csv'

fleet = Fleet(avg_dir=avg_path,cmp_dir=compressed_path, 
              yaw_dir=yaw_path, 
              oem_powercurves_path=oem_powercurves_path, 
              single_plant='PDK') # Remove this if you want to run more than 1 site

powercurves = fleet.get_powercurves()
powercurve_distributions = fleet.get_powercurve_distributions()
daily_severity_scores = fleet.daily_severity_scores
daily_severity_scores.columns = [x.replace("-LOST-ENERGY","-SEVERITY") for x in daily_severity_scores.columns]
daily_efficiencies =fleet.get_daily_efficiency()
daily_lost_energy = fleet.get_daily_lost_energy()
daily_lost_revenue = fleet.get_daily_lost_revenue()
daily_means = fleet.get_daily_mean()
daily_severity_scores = pd.concat([daily_severity_scores,daily_efficiencies,daily_lost_energy,daily_lost_revenue,daily_means], axis=1)

daily_severity_scores.to_csv('../southernoperations/AIID/fleet_daily_severity_scores_stage.csv')
powercurves.to_csv('../southernoperations/AIID/power_curve.csv')
powercurve_distributions.to_csv('../southernoperations/AIID/power_curve_counts.csv')

Fleet 125: Processing PDK...
Fleet 391: Calculating Power Curves for PDK
[WindFarm 496] processing severity PDK, Xfrm_Aux_Temp...
[WindFarm 496] processing severity PDK, Ctrl_Gnd_Temp...
[WindFarm 496] processing severity PDK, Ctrl_Hub_Temp...
[WindFarm 496] processing severity PDK, Ctrl_Top_Temp...
[WindFarm 496] processing severity PDK, Ctrl_VCP_Temp...
[WindFarm 496] processing severity PDK, Gbx_CoolingWater_Temp...
[WindFarm 496] processing severity PDK, Gbx_Oil_Temp...
[WindFarm 496] processing severity PDK, Main_Brg_Temp...
[WindFarm 496] processing severity PDK, Gen_Brg_DE_Temp...
[WindFarm 496] processing severity PDK, Gen_Brg_NDE_Temp...
[WindFarm 496] processing severity PDK, Gen_CoolingFluid_Temp...
[WindFarm 496] processing severity PDK, Gen_Windings_PhaseA_Temp...
[WindFarm 496] processing severity PDK, Gen_Windings_PhaseB_Temp...
[WindFarm 496] processing severity PDK, Gen_Windings_PhaseC_Temp...
[WindFarm 496] processing severity PDK, Gbx_Brg_HighSpd_Temp...
[WindFarm 49

# Process Fleet Output
This finalizes the processed data for use in the UI performance and reliability area 

In [2]:
import pandas as pd 

treemap_data = pd.read_csv('../southernoperations/AIID/fleet_daily_severity_scores_stage.csv', index_col=[0], parse_dates = [0])
try:
    treemap_data = treemap_data[~treemap_data.index.to_series().str.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}')]
except:
    pass

treemap_data.index =pd.to_datetime(treemap_data.index)

type_func = lambda x: '-'.join(x.split('-')[2:])
col_suffixes = sorted(list(set([type_func(x) for x in treemap_data.columns])))

temp_treemap_data= treemap_data[[x for x in treemap_data.columns if not any(y in x for y in ['KW','SIMPLE-EFFICIENCY'])]]

temp_treemap_data.to_csv('../southernoperations/AIID/treemap_data.csv')

efficiency_treemap = treemap_data[[x for x in treemap_data.columns if any(y in type_func(x) or y.endswith('_mean') for y in ['KW',
                                                                                           'EFFICIENCY',
                                                                                           'SEVERITY',
                                                                                           'LOST-ENERGY',
                                                                                           'LOST-REVENUE'])]]
efficiency_treemap.to_csv('../southernoperations/AIID/treemap_data_simple_efficiency.csv')

yaw_error_treemap_data = treemap_data[[x for x in treemap_data.columns if any(y in x for y in ['YAW-ERROR'])]]
yaw_error_treemap_data.to_csv('../southernoperations/AIID/radial_yaw_error.csv')

print("Data output to southernoperations/AIID")

Data output to southernoperations/AIID


# Process Fault Analysis Data Sets

Processes raw 10 minute average and compressed fault code data into data ready to be consumed  by the UI in the fault analysis area

In [3]:
import sys
import os
import datetime
import pandas as pd

sys.path.append('..')
from Model.Fault import FaultAnalysis

def add_file_to_project_dict(file_path, project_dict):
    """
    Opens a CSV file, identifies the project based on the second column's name, 
    and adds the file path to a dictionary keyed by the project.

    Args:
    file_path (str): The path to the CSV file.
    project_dict (dict): The dictionary to update with the file path.

    Returns:
    dict: Updated dictionary with the file path added under the appropriate project key.
    """
    # Read the CSV file
    df = pd.read_csv(file_path, nrows=1)
    
    # Extract the project name from the second column
    project_name = df.columns[1].split('-')[0]

    # Add the file path to the dictionary under the project name key
    project_dict[project_name] = file_path
    
    return project_dict

projects = [
              'BR2', 
              'BTH', 
              'CFW', 
              'DHW', 
              'GNT', 
              'GNP', 
              'GSW', 
              'KAY', 
              'PDK', 
              'RDG', 
              'SFK', 
              'SKO', 
              'TBF', 
              'WAK', 
              'WHE'
            ]

fault_analysis_objects = {}
project_cmp_dict = {}
project_avg_dict = {}

cmp_dir = '../southernoperations/AIID/CMP'
avg_dir = '../southernoperations/AIID/AVG'

for file in os.listdir(cmp_dir):
    this_path = os.path.join(cmp_dir, file)
    project_cmp_dict = add_file_to_project_dict(this_path, project_dict=project_cmp_dict)

for file in os.listdir(avg_dir):
    this_path = os.path.join(avg_dir, file)
    project_avg_dict = add_file_to_project_dict(this_path, project_dict=project_avg_dict)

for project in projects:
    
    fault_data_path = project_cmp_dict[project]
    
    avg_data_path = project_avg_dict[project]
    
    print('Processing', project)
    
    cmp_data = pd.read_csv(fault_data_path, low_memory=False)
    avg_data = pd.read_csv(avg_data_path, index_col=[0], parse_dates=[0])

    downtime = FaultAnalysis(project=project, cmp_data=cmp_data, avg_data=avg_data)

    downtime_data = downtime.reshaped_data
    
    #put each object in memory for use after run 
    fault_analysis_objects[project] = downtime
    if downtime_data.shape[0] > 0:
        output_data = downtime_data.loc[downtime_data['StartDateTime'] > '8/1/2023'].dropna()
        output_data.to_csv(f'../southernoperations/AIID/Processed Faults/{project}_downtime_lost_energy.csv', index=False)

        # daily file output
        daily_data = downtime.daily_downtime()
        daily_data['FaultCount'] = downtime.daily_fault_count()["FaultCount"]
        daily_data['LostEnergy'] = downtime.daily_lost_energy()['LostEnergy']
        daily_data['LostRevenue'] = downtime.daily_lost_revenue()['LostRevenue']

        daily_data.to_csv(f'../southernoperations/AIID/Processed Faults/{project}-daily-fault.csv', index=False)

        print('Fault data transformed and output to "southernoperations/AIID/Processed Faults"')
    else:
        print("The downtime dataset is empty")


Processing BR2
       BR2-K001
       BR2-K002
       BR2-K003
       BR2-K004
       BR2-K005
       BR2-K006
       BR2-K007
       BR2-K008
       BR2-K009
       BR2-K010
       BR2-K011
       BR2-K012
       BR2-K013
       BR2-K014
       BR2-K015
       BR2-K016
       BR2-K017
       BR2-K018
       BR2-K019
       BR2-K020
Fault data transformed and output to "southernoperations/AIID/Processed Faults"
Processing BTH
       BTH-T001
       BTH-T002
       BTH-T003
       BTH-T004
       BTH-T005
       BTH-T006
       BTH-T007
       BTH-T008
       BTH-T009
       BTH-T010
       BTH-T011
       BTH-T012
       BTH-T013
       BTH-T014
       BTH-T015
       BTH-T016
       BTH-T017
       BTH-T018
       BTH-T019
       BTH-T020
       BTH-T021
       BTH-T022
       BTH-T023
       BTH-T024
       BTH-T025
       BTH-T026
       BTH-T027
       BTH-T028
       BTH-T029
       BTH-T030
       BTH-T031
       BTH-T032
       BTH-T033
       BTH-T034
       BTH-T035
BTH-T036 i

# Concatenate Fault Metric Files 

In [ ]:
import pandas as pd
import os

# Define the directory path
directory_path = '../southernoperations/AIID/Processed Faults/'
search_strings = ['daily-fault.csv','downtime_lost_energy.csv']
output_file_names = ['../southernoperations/AIID/daily_turbine_fault.csv','../southernoperations/AIID/downtime_lost_energy.csv']

for i, search_str in enumerate(search_strings):
    output_file_name = output_file_names[i]
    
    # Get all files in the directory
    all_files = os.listdir(directory_path)

    # Filter files that contain the words 'downtime_lost_energy.csv'
    filtered_files = [f for f in all_files if search_str in  f]

    # Initialize an empty list to store DataFrames
    dfs = []

    # Read and store each file in the list
    for file in filtered_files:
        file_path = os.path.join(directory_path, file)
        dfs.append(pd.read_csv(file_path))

    # Concatenate all DataFrames
    combined_df = pd.concat(dfs, ignore_index=True)

    # Save the concatenated DataFrame to a new file
    output_file = output_file_name
    combined_df =combined_df.dropna().reset_index()
    if "Unnamed: 0" in combined_df.columns:
        combined_df = combined_df.drop("Unnamed: 0", axis = 1)
    combined_df.to_csv(output_file, index=False)

    print(f"Concatenated {len(filtered_files)} files and saved to {output_file}")



# Generate removed data report

In [ ]:
import pandas as pd
all_frames = []
for wf_name, wf_obj in fleet.windfarms.items():
    
    for component,obj in wf_obj.components.items():
        print(wf_name, component)
        if hasattr(obj,'calculate_data_removal_stats'):
            this_stats = obj.calculate_data_removal_stats()
            all_frames.append(this_stats)
            

pd.concat(all_frames).to_csv('../southernoperations/AIID/recovery_report.csv')

In [ ]:
# Insert the line below into the terminal to output link to where to visualize the results above
# python index.py

## Gantt chart for project proting to databricks 

In [23]:
import pandas as pd
import plotly.express as px

# Define the task data for the Gantt chart
data = [
    {"Task": "Data Inventory Pre-Check", "Start": "2025-01-27", "Duration": 1, "Dataset": None},
    {"Task": "Data Loading", "Start": None, "Duration": 2, "Dataset": "Raw Data Table"},
    {"Task": "Parameter Management", "Start": None, "Duration": 2, "Dataset": None},
    {"Task": "Gradient Cleaning", "Start": None, "Duration": 1, "Dataset": "Gradient Filtered Tables"},
    {"Task": "Range Cleaning", "Start": None, "Duration": 1, "Dataset": "Range Filtered Tables"},
    {"Task": "Online Filtering", "Start": None, "Duration": 2, "Dataset": "Filtered Online Data Table"},
    {"Task": "Final Clean Data", "Start": None, "Duration": 1, "Dataset": "Final Cleaned Dataset"},
    {"Task": "Data Recovery", "Start": None, "Duration": 1, "Dataset": None},
    {"Task": "Daily Efficiency", "Start": None, "Duration": 1, "Dataset": "Efficiency Metrics"},
    {"Task": "Daily Lost Energy", "Start": None, "Duration": 1, "Dataset": "Lost Energy Metrics"},
    {"Task": "Daily Lost Revenue", "Start": None, "Duration": 1, "Dataset": "Lost Revenue Metrics"},
    {"Task": "Daily Severity", "Start": None, "Duration": 2, "Dataset": "Severity Metrics"},
    {"Task": "Daily Mean", "Start": None, "Duration": 1, "Dataset": "Daily Mean Dataset"},
    {"Task": "Power Curves", "Start": None, "Duration": 2, "Dataset": "Power Curves Table"},
    {"Task": "Power Curve Distributions", "Start": None, "Duration": 1, "Dataset": "Power Curve Distribution"},
    {"Task": "Reshaped Data (Faults Master Table)", "Start": None, "Duration": 4, "Dataset": "Faults Master Table"},
    {"Task": "Daily Downtime", "Start": None, "Duration": 1, "Dataset": "Downtime Metrics"},
    {"Task": "Daily Fault Count", "Start": None, "Duration": 1, "Dataset": "Fault Count Metrics"},
    {"Task": "Daily Lost Energy (Faults)", "Start": None, "Duration": 1, "Dataset": "Lost Energy Faults"},
    {"Task": "Daily Lost Revenue (Faults)", "Start": None, "Duration": 1, "Dataset": "Lost Revenue Faults"},
]

# Helper function to calculate task segments (split tasks over weekends)
def split_task(task):
    segments = []
    start_date = pd.Timestamp(task["Start"])
    duration = task["Duration"]

    while duration > 0:
        days_to_weekend = 4 - start_date.weekday()
        work_days = min(days_to_weekend + 1, duration) if start_date.weekday() < 5 else 0
        end_date = start_date + pd.Timedelta(days=work_days - 1)

        if work_days > 0:
            segments.append({
                "Task": task["Task"],
                "Start": start_date,
                "End": end_date + pd.Timedelta(hours=12) if start_date == end_date else end_date,
                "Dataset": task["Dataset"] if len(segments) == 0 else None
            })

        duration -= work_days
        start_date = end_date + pd.Timedelta(days=1)
        while start_date.weekday() >= 5:
            start_date += pd.Timedelta(days=1)

    return segments

# Assign start dates and split tasks over weekends
current_start = pd.Timestamp("2025-01-13")
split_tasks = []
for task in data:
    if not task["Start"]:
        task["Start"] = current_start
    split_tasks.extend(split_task(task))
    current_start = split_tasks[-1]["End"] + pd.Timedelta(days=1)
    while current_start.weekday() >= 5:
        current_start += pd.Timedelta(days=1)

# Create a DataFrame
df = pd.DataFrame(split_tasks)

# Generate the Gantt chart
fig = px.timeline(
    df,
    x_start="Start",
    x_end="End",
    y="Task",
    title="Gantt Chart for Databricks Porting with Updated Schedule",
    labels={"Task": "Tasks"},
    hover_data=["Dataset"],
)

# Extend the x-axis to avoid clipping
fig.update_layout(
    xaxis=dict(title="Timeline", range=[df["Start"].min(), df["End"].max() + pd.Timedelta(days=4)]),
    yaxis=dict(title="Tasks", autorange="reversed"),
    height=1200,
    width=1400,
    template="plotly_white",
)

# Add annotations for datasets, offset to the right of the leftmost edge
annotations = []

offset = pd.Timedelta(days=2)  # Offset the annotations slightly to the right

for task_name in df["Task"].unique():
    task_segments = df[df["Task"] == task_name]
    rightmost_segment = task_segments[task_segments["Dataset"].notnull()]
    if not rightmost_segment.empty:
        annotations.append(
            {
                "x": df["Start"].min() + offset,  # Offset slightly to the right
                "y": rightmost_segment["Task"].values[0],
                "text": f"QA: {rightmost_segment['Dataset'].values[0]}",
                "showarrow": False,
                "font": {"size": 10, "color": "black"},
                "align": "left",
                "xanchor": "left",
                "xref": "x",
                "yref": "y",
            }
        )

fig.update_layout(annotations=annotations)

# Show the chart
fig.show()
